In [1]:
import numpy as np
import pickle
import sys
import argparse
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
np.random.seed(1337)
import torch.backends.cudnn as cudnn
import torch, torch.nn as nn, math, torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
#prepare dataset
import torch.utils.data
Batch_size = 10
Epochs = 200
validation_split = 0.2
shuffle = True

def data_load(model, batch_size = 10, valid_size = 0.2):
    # train, validation, test data 
    # question 1, if the train data and test data are from the same data distribution, will this be problem that the model overfit to the dataset
    
    with open('./input/'+model+'.pickle', 'rb') as handle:
                (train_x, train_y, test_x, test_y, maxlen, train_len, test_len) = pickle.load(handle)
    train_len = np.array(train_len)
    test_len = np.array(test_len)
    
    train_x = torch.from_numpy(train_x).double()
    train_y = torch.from_numpy(train_y).double()
    test_x = torch.from_numpy(test_x).double()
    test_y = torch.from_numpy(test_y).double()
    train_len = torch.from_numpy(train_len)
    test_len = torch.from_numpy(test_len)
    
    num_train = len(train_x) 
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    random_seed = 2018
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
        
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    train = torch.utils.data.TensorDataset(train_x, train_y, train_len)
    test = torch.utils.data.TensorDataset(test_x, test_y, test_len)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size= batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(train, batch_size= batch_size, sampler=valid_sampler)
    test_loader = torch.utils.data.DataLoader(test, batch_size= batch_size, shuffle= True)
    return train_loader, valid_loader, test_loader


In [5]:
class Unimodel(nn.Module):
    def __init__(self, input_size, hidden_size = 300, out_size = 100):
        super(Unimodel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=1,batch_first=True, bidirectional=True)
        self.dropout1 = nn.Dropout(p = 0.6)
        self.fc1 = nn.Linear(600,out_size) 
        self.dropout2 = nn.Dropout(p = 0.9)
        self.tanh = nn.Hardtanh(-1,1)
        self.fc2 = nn.Linear(out_size, 2) 
       
    def forward(self, inputs, sequence_len): # input is 10 * 63 * 100
        # training details:
        pack = torch.nn.utils.rnn.pack_padded_sequence(inputs, sequence_len, batch_first=True)
        hidden = (
        Variable(torch.zeros(2, inputs.size(0), self.hidden_size),requires_grad=True),
        Variable(torch.zeros(2, inputs.size(0), self.hidden_size),requires_grad=True))
        #print(pack)
        out, hidden = self.lstm(pack, hidden)
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        #print(unpacked.size())
        output = self.dropout2(self.tanh(unpacked)) # apply drop out
        output = self.dropout2(self.tanh(self.fc1(output))) # 100
        output = self.fc2(output) # 2
        #print(output)
        return output
    

In [ ]:
# training unimodel, use the index to trace the loss function.
epochs = 200
criterion = nn.CrossEntropyLoss()
models = ['text', 'audio', 'video']
input_size = {'text': 100, 'audio': 73, 'video':100}
use_gpu = torch.cuda.is_available()
print(use_gpu)
def sorted_sequence(input_x,sequences,y):
    # return the sorted sequence based on input_x
    diction = {}
    for i in range(int(sequences.size(0))):
        diction[i] = sequences[i]
    new_sequence = []
    new_x = torch.zeros(input_x.size(0),input_x.size(1),input_x.size(2))
    new_y = torch.zeros(y.size(0),y.size(1))
    count = 0
    for i, value in sorted(diction.items(), key=lambda x:x[1], reverse = True):
        new_sequence.append(int(value))
        new_x[count] = input_x[i]
        new_y[count] = y[i]
        count +=1
    return new_x,new_sequence,new_y

def cast_y(max_len, y):
    new_y = torch.zeros(y.size(0), max_len)
    for i in range(y.size(0)):
        new_y[i] = y[i][:max_len]
    return new_y

def one_hot(train_y):
    maxlabel = 1
    new_y = torch.zeros((train_y.size(0),train_y.size(1), maxlabel+1))
    for i in range(int(train_y.size(0))):
        for j in range(int(train_y.size(1))):
            new_y[i,j,int(train_y[i,j])] =1
    return new_y
loss_list = []
for mode in models:
    model = Unimodel(input_size[mode])
    optimizer = optim.Adagrad(params = model.parameters(), lr = 0.01)
    running_loss = 0
    for epoch in range(epochs):
        train_loader, valid_loader, test_loader = data_load(mode, 10, 0.2)
        batch = 0
        for e, data in enumerate(train_loader):
            optimizer.zero_grad()
            input_x, y, sequence_len = data[0], data[1], data[2]
            input_x, sequence_len, y = sorted_sequence(input_x,sequence_len, y)
            input_x = Variable(input_x.cuda(), requires_grad=True)
            y = cast_y(sequence_len[0], y)
            y = Variable(y.view(-1).cuda())
            predict_y = model(input_x, sequence_len)
            predict_y = Variable(predict_y.view(-1,2),requires_grad=True)
            #print(predict_y.view(-1,2).size(), y.view(-1,2).size())
            loss = criterion(predict_y, y.long())
            print(loss)
            loss.backward()
            optimizer.step()
            running_loss += loss
            batch += 1
        #if(epoch %10 == 0):
        print("epoch %d loss % f" %(epoch, running_loss/batch))
        loss_list.append(running_loss/batch)
        running_loss = 0
        batch = 0
    save_model(model, optimizer, os.path.join(save_dir, '%03d.ckpt' % epoch))  

In [136]:
class Multimodel(nn.Module):
    def __init__(self, input_size, hidden_size = 300, out_size = 100):
        super(Unimodel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True)
        self.dropout1 = nn.Dropout(p = 0.4)
        self.fc1 = nn.Linear(hidden_size,out_size) 
        self.dropout2 = nn.Dropout(p = 0.9)
        self.tanh = nn.Hardtanh(-1,1)
        self.fc2 = nn.Linear(out_size, 2) 
        self.output = nn.Softmax()
        
    def forward(self, inputs): 
        hidden_vect = (
        Variable(torch.zeros(2, 1, self.hidden_size)),
        Variable(torch.zeros(2, 1, self.hidden_size)))
        output, hidden_vect = self.lstm(Variable(inputs), hidden_vect)
        output = self.dropout2(self.tanh(output)) # apply drop out
        output = self.dropout2(self.tanh(self.fc1(output))) # 100
        output = self.softmax(self.fc2(output)) # 2
        return output
    